# Import The Libraries

In [1]:
import numpy as np
import cv2
import pandas as pd

In [2]:
from matplotlib import pyplot as plt
import os

In [3]:
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd

In [4]:
import gif2numpy

# Building Datasets

In [5]:
train_path = "D:\\Data sets\\DRIVE\\training"
train_ids = next(os.walk(train_path + "\\images"))[2]
gt_ids = next(os.walk(train_path + "\\1st_manual"))[2]

In [6]:
train_ids, gt_ids

(['21_training.tif',
  '22_training.tif',
  '23_training.tif',
  '24_training.tif',
  '25_training.tif',
  '26_training.tif',
  '27_training.tif',
  '28_training.tif',
  '29_training.tif',
  '30_training.tif',
  '31_training.tif',
  '32_training.tif',
  '33_training.tif',
  '34_training.tif',
  '35_training.tif',
  '36_training.tif',
  '37_training.tif',
  '38_training.tif',
  '39_training.tif',
  '40_training.tif'],
 ['21_manual1.gif',
  '22_manual1.gif',
  '23_manual1.gif',
  '24_manual1.gif',
  '25_manual1.gif',
  '26_manual1.gif',
  '27_manual1.gif',
  '28_manual1.gif',
  '29_manual1.gif',
  '30_manual1.gif',
  '31_manual1.gif',
  '32_manual1.gif',
  '33_manual1.gif',
  '34_manual1.gif',
  '35_manual1.gif',
  '36_manual1.gif',
  '37_manual1.gif',
  '38_manual1.gif',
  '39_manual1.gif',
  '40_manual1.gif'])

In [7]:
def originalImg(img,df):
    img2 = img.reshape(-1)
    df['Original_Image'] = img2
    return df,img2

In [8]:
def gaborFilter(img2,df):
    #Generate Gabor features
    num = 1  #To count numbers up in order to give Gabor features a lable in the data frame
    kernels = []
    for theta in range(2):   #Define number of thetas
        theta = theta / 4. * np.pi
        for sigma in (1, 3):  #Sigma with 1 and 3
            for lamda in np.arange(0, np.pi, np.pi / 4):   #Range of wavelengths
                for gamma in (0.05, 0.5):   #Gamma values of 0.05 and 0.5


                    gabor_label = 'Gabor' + str(num)  #Label Gabor columns as Gabor1, Gabor2, etc.
                    # print(gabor_label)
                    ksize=5
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                    kernels.append(kernel)
                    #Now filter the image and add values to a new column 
                    fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
                    #print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                    num += 1  #Increment for gabor column label
    return df

In [9]:
def cannyEdge(df,img):
    #CANNY EDGE
    edges = cv2.Canny(img, 100,200)   #Image, min and max values
    edges1 = edges.reshape(-1)
    df['Canny_Edge'] = edges1 #Add column to original dataframe
    return df

In [10]:
def the4edgeDetectors(df,img):
    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df['Roberts'] = edge_roberts1
    
    #SOBEL
    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df['Sobel'] = edge_sobel1
    
    #SCHARR
    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df['Scharr'] = edge_scharr1
    
    #PREWITT
    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['Prewitt'] = edge_prewitt1
    
    return df

In [11]:
def gmv(df,img):
    #GAUSSIAN with sigma=3

    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df['Gaussian s3'] = gaussian_img1

    #GAUSSIAN with sigma=7
    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df['Gaussian s7'] = gaussian_img3

    #MEDIAN with sigma=3
    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    df['Median s3'] = median_img1

    #VARIANCE with size=3
    variance_img = nd.generic_filter(img, np.var, size=3)
    variance_img1 = variance_img.reshape(-1)
    df['Variance s3'] = variance_img1  #Add column to original dataframe
    return df

In [12]:
def gtImage(gt_path,df):
    frames, exts, image_specs = gif2numpy.convert(gt_path)
    frames = np.array(frames)
    labeled_img = frames.reshape(584, 565,3)
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
    labeled_img1 = labeled_img.reshape(-1)
    df['Labels'] = labeled_img1
    return df

In [13]:
def gtImage(gt_path,df):
    frames, exts, image_specs = gif2numpy.convert(gt_path)
    frames = np.array(frames)
    labeled_img = frames.reshape(584, 565,3)
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
    labeled_img1 = labeled_img.reshape(-1)
    df['Labels'] = labeled_img1
    return df

In [14]:
image_dataset = pd.DataFrame()

In [16]:
for _id in range(20):
    image = train_ids[_id]
    gt_image = gt_ids[_id]
    
    df = pd.DataFrame()  #Temporary data frame for each Image
    
    path = train_path + "\\images\\" + image
    print(path,"Loading...")
    
    img = cv2.imread(path)
    img = img[:,:,1]
    df,img2 = originalImg(img,df)
    df['Image_Name'] = image
    df = gaborFilter(img2,df)
    
    df = cannyEdge(df,img)
    
    df = the4edgeDetectors(df,img)
    
    df = gmv(df,img)
    
    gt_path = train_path + "\\1st_manual\\" + gt_image
    print(gt_path)
    df = gtImage(gt_path,df)
    
    image_dataset = pd.concat([image_dataset,df])
    print(image_dataset.shape)

D:\Data sets\DRIVE\training\images\21_training.tif Loading...
D:\Data sets\DRIVE\training\1st_manual\21_manual1.gif
<class 'str'>


KeyError: -1

In [16]:
image_dataset.head()

,Original_Image,Image_Name,Gabor1,Gabor2,Gabor3,Gabor4,Gabor5,Gabor6,Gabor7,Gabor8,...,Canny_Edge,Roberts,Sobel,Scharr,Prewitt,Gaussian s3,Gaussian s7,Median s3,Variance s3,Labels
0,0,21_training.tif,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,1,0,0,0
1,0,21_training.tif,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,1,0,0,0
2,0,21_training.tif,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,1,0,0,0
3,0,21_training.tif,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,1,0,0,0
4,0,21_training.tif,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0,1,0,0,0


In [17]:
image_dataset.tail()

,Original_Image,Image_Name,Gabor1,Gabor2,Gabor3,Gabor4,Gabor5,Gabor6,Gabor7,Gabor8,...,Canny_Edge,Roberts,Sobel,Scharr,Prewitt,Gaussian s3,Gaussian s7,Median s3,Variance s3,Labels
329955,0,40_training.tif,0,0,0,0,0,0,0,0,...,0,0.000000,0.019460,0.019439,0.019499,4,5,0,11,0
329956,0,40_training.tif,0,0,1,0,0,0,0,0,...,0,0.000000,0.018024,0.017678,0.018486,4,5,0,9,0
329957,0,40_training.tif,0,0,1,0,0,0,0,0,...,0,0.003922,0.018236,0.018579,0.017780,4,5,1,9,0
329958,1,40_training.tif,0,0,1,1,0,0,0,0,...,0,0.003922,0.018024,0.017678,0.018486,4,5,1,10,0
329959,0,40_training.tif,0,0,1,1,0,0,0,0,...,0,0.000000,0.018833,0.019025,0.018579,4,5,1,10,0


# Train & Test 

In [19]:
X = image_dataset.drop(labels = ["Image_Name","Labels"], axis=1)
X.head()

,Original_Image,Gabor1,Gabor2,Gabor3,Gabor4,Gabor5,Gabor6,Gabor7,Gabor8,Gabor9,...,Gabor32,Canny_Edge,Roberts,Sobel,Scharr,Prewitt,Gaussian s3,Gaussian s7,Median s3,Variance s3
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,1,0,0


In [20]:
Y = image_dataset["Labels"]
Y.head()

0    0
1    0
2    0
3    0
4    0
Name: Labels, dtype: uint8

# Building The 20 Models

In [20]:
329960 * 2

659920

In [21]:
584 * 565

329960

In [15]:
from sklearn.model_selection import train_test_split

In [23]:
import xgboost as xgb

In [24]:
from sklearn import metrics

In [25]:
from sklearn.metrics import classification_report

In [26]:
import pickle

In [30]:
def buildModel(file,data):
    X = data.drop(labels = ["Image_Name","Labels"], axis=1)
    Y = data["Labels"]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=20)
    
    xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, seed=123)
    eval_set = [(X_train, y_train), (X_test, y_test)]
    
    xg_cl.fit(X_train, y_train, eval_metric=["error"], eval_set=eval_set, verbose=True)
    results = xg_cl.evals_result()
    prediction_test = xg_cl.predict(X_test)
    print ("Accuracy on the Original Image = ", metrics.accuracy_score(y_test, prediction_test))
    
    print(classification_report(y_test, prediction_test))
    print("Macro Avg:",classification_report(y_test, prediction_test).split()[21])
    
    filelName = "models\\" + str(file.split('.')[0])
    pickle.dump(xg_cl, open(filelName, 'wb'))
    
    

In [31]:
start = 0
end = 584 * 565
size = 584 * 565

In [32]:
start = 0
end = 584 * 565
size = 584 * 565
for _id in range(len(train_ids)):
    print(train_ids[_id],"Loading...")
    demo = image_dataset.iloc[start:end,:]
    print(demo.iloc[:2,:5])
    print(demo.iloc[-2:,:5])
    
    buildModel(train_ids[_id],demo)  
    
    start += size
    end += size
    print(train_ids[_id],"Finished...")
    print("______________________________________________________________________________")

21_training.tif Loading...
   Original_Image       Image_Name  Gabor1  Gabor2  Gabor3
0               0  21_training.tif       0       0       0
1               0  21_training.tif       0       0       0
        Original_Image       Image_Name  Gabor1  Gabor2  Gabor3
329958               1  21_training.tif       0       0       2
329959               1  21_training.tif       0       0       2


C:\Users\Shirsendu\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-error:0.03935	validation_1-error:0.04004
[1]	validation_0-error:0.03618	validation_1-error:0.03665
[2]	validation_0-error:0.03481	validation_1-error:0.03524
[3]	validation_0-error:0.03328	validation_1-error:0.03411
[4]	validation_0-error:0.03273	validation_1-error:0.03363
[5]	validation_0-error:0.03180	validation_1-error:0.03282
[6]	validation_0-error:0.03139	validation_1-error:0.03257
[7]	validation_0-error:0.03130	validation_1-error:0.03248
[8]	validation_0-error:0.03099	validation_1-error:0.03214
[9]	validation_0-error:0.03062	validation_1-error:0.03162
[10]	validation_0-error:0.03048	validation_1-error:0.03137
[11]	validation_0-error:0.03025	validation_1-error:0.03115
[12]	validation_0-error:0.03025	validation_1-error:0.03113
[13]	validation_0-error:0.02991	validation_1-error:0.03095
[14]	validation_0-error:0.02962	validation_1-error:0.03078
[15]	validation_0-error:0.02947	validation_1-error:0.03059
[16]	validation_0-error:0.02923	validation_1-error:0.03050
[17]	va

C:\Users\Shirsendu\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-error:0.04762	validation_1-error:0.04870
[1]	validation_0-error:0.04349	validation_1-error:0.04468
[2]	validation_0-error:0.04307	validation_1-error:0.04502
[3]	validation_0-error:0.04149	validation_1-error:0.04294
[4]	validation_0-error:0.04095	validation_1-error:0.04235
[5]	validation_0-error:0.04061	validation_1-error:0.04214
[6]	validation_0-error:0.03992	validation_1-error:0.04163
[7]	validation_0-error:0.03957	validation_1-error:0.04142
[8]	validation_0-error:0.03941	validation_1-error:0.04106
[9]	validation_0-error:0.03912	validation_1-error:0.04053
[10]	validation_0-error:0.03880	validation_1-error:0.04025
[11]	validation_0-error:0.03871	validation_1-error:0.04002
[12]	validation_0-error:0.03830	validation_1-error:0.03950
[13]	validation_0-error:0.03800	validation_1-error:0.03916
[14]	validation_0-error:0.03784	validation_1-error:0.03905
[15]	validation_0-error:0.03769	validation_1-error:0.03889
[16]	validation_0-error:0.03757	validation_1-error:0.03901
[17]	va

...

[0]	validation_0-error:0.05172	validation_1-error:0.05361
[1]	validation_0-error:0.04729	validation_1-error:0.04836
[2]	validation_0-error:0.04661	validation_1-error:0.04739
[3]	validation_0-error:0.04530	validation_1-error:0.04656
[4]	validation_0-error:0.04415	validation_1-error:0.04541
[5]	validation_0-error:0.04357	validation_1-error:0.04470
[6]	validation_0-error:0.04272	validation_1-error:0.04447
[7]	validation_0-error:0.04248	validation_1-error:0.04402
[8]	validation_0-error:0.04202	validation_1-error:0.04382
[9]	validation_0-error:0.04167	validation_1-error:0.04339
[10]	validation_0-error:0.04118	validation_1-error:0.04281
[11]	validation_0-error:0.04079	validation_1-error:0.04258
[12]	validation_0-error:0.04084	validation_1-error:0.04215
[13]	validation_0-error:0.04051	validation_1-error:0.04203
[14]	validation_0-error:0.04018	validation_1-error:0.04190
[15]	validation_0-error:0.03985	validation_1-error:0.04179
[16]	validation_0-error:0.03956	validation_1-error:0.04169
[17]	va

C:\Users\Shirsendu\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-error:0.04863	validation_1-error:0.04821
[1]	validation_0-error:0.04567	validation_1-error:0.04523
[2]	validation_0-error:0.04455	validation_1-error:0.04442
[3]	validation_0-error:0.04390	validation_1-error:0.04368
[4]	validation_0-error:0.04339	validation_1-error:0.04348
[5]	validation_0-error:0.04352	validation_1-error:0.04353
[6]	validation_0-error:0.04299	validation_1-error:0.04307
[7]	validation_0-error:0.04260	validation_1-error:0.04282
[8]	validation_0-error:0.04222	validation_1-error:0.04245
[9]	validation_0-error:0.04203	validation_1-error:0.04235
[10]	validation_0-error:0.04175	validation_1-error:0.04209
[11]	validation_0-error:0.04135	validation_1-error:0.04203
[12]	validation_0-error:0.04113	validation_1-error:0.04182
[13]	validation_0-error:0.04100	validation_1-error:0.04161
[14]	validation_0-error:0.04076	validation_1-error:0.04156
[15]	validation_0-error:0.04067	validation_1-error:0.04140
[16]	validation_0-error:0.04065	validation_1-error:0.04143
[17]	va

C:\Users\Shirsendu\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-error:0.04311	validation_1-error:0.04336
[1]	validation_0-error:0.03949	validation_1-error:0.03994
[2]	validation_0-error:0.03789	validation_1-error:0.03832
[3]	validation_0-error:0.03725	validation_1-error:0.03775
[4]	validation_0-error:0.03647	validation_1-error:0.03723
[5]	validation_0-error:0.03591	validation_1-error:0.03676
[6]	validation_0-error:0.03496	validation_1-error:0.03565
[7]	validation_0-error:0.03470	validation_1-error:0.03525
[8]	validation_0-error:0.03425	validation_1-error:0.03502
[9]	validation_0-error:0.03391	validation_1-error:0.03473
[10]	validation_0-error:0.03363	validation_1-error:0.03442
[11]	validation_0-error:0.03352	validation_1-error:0.03431
[12]	validation_0-error:0.03318	validation_1-error:0.03405
[13]	validation_0-error:0.03304	validation_1-error:0.03417
[14]	validation_0-error:0.03299	validation_1-error:0.03426
[15]	validation_0-error:0.03284	validation_1-error:0.03394
[16]	validation_0-error:0.03268	validation_1-error:0.03369
[17]	va

# Comparing the Models

In [33]:
model_path = "C:\\Users\\Shirsendu\\A Project\\models"
model_ids = next(os.walk(model_path))[2]
model_ids

['21_training',
 '22_training',
 '23_training',
 '24_training',
 '25_training',
 '26_training',
 '27_training',
 '28_training',
 '29_training',
 '30_training',
 '31_training',
 '32_training',
 '33_training',
 '34_training',
 '35_training',
 '36_training',
 '37_training',
 '38_training',
 '39_training',
 '40_training']

In [34]:
from tqdm import tqdm

In [35]:
def modelPerformance(modelName):
    acc_set = []
    path = "models\\" + modelName
    model = pickle.load(open(path,'rb'))
    start = 0
    end = 584 * 565
    size = 584 * 565
    for _id in range(len(train_ids)):
        print()
        print(train_ids[_id],"Loading...")
        demo = image_dataset.iloc[start:end,:]
        
        x = demo.drop(labels = ["Image_Name","Labels"], axis=1)
        y = demo["Labels"]
        
        prediction_test = model.predict(x)
        
        acc = metrics.accuracy_score(y, prediction_test)
        print ("Accuracy = ", acc)
        print("Macro Avg:",classification_report(y, prediction_test).split()[21])
        
        acc_set.append(classification_report(y, prediction_test).split()[21])

        start += size
        end += size
        print(train_ids[_id],"Finished...")
        print("* * * * * * * * * * * * * * * * * * * *")
    return acc_set


In [36]:
acc_total = []

In [37]:
acc_total.append(model_ids)

In [38]:
for mod in tqdm(model_ids):
#     mod += "Model" 
    print(mod, "Model starting...")
    mod_acc = modelPerformance(mod)
    acc_total.append(mod_acc)
    print()
    print("________________________________________________________________________________________")

  0%|          | 0/20 [00:00<?, ?it/s]

21_training Model starting...

21_training.tif Loading...
Accuracy =  0.9762516668687113
Macro Avg: 0.91
21_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

22_training.tif Loading...
Accuracy =  0.8345799490847375
Macro Avg: 0.66
22_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

23_training.tif Loading...
Accuracy =  0.9184810280033944
Macro Avg: 0.75
23_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

24_training.tif Loading...
Accuracy =  0.9510971026791126
Macro Avg: 0.86
24_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

25_training.tif Loading...
Accuracy =  0.6820432779730876
Macro Avg: 0.55
25_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

26_training.tif Loading...
Accuracy =  0.8993665898896835
Macro Avg: 0.49
26_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

27_training.tif Loading...
Accuracy =  0.95526427445751
Macro Avg: 0.86
27_training.tif Finished...
* * * * * *

  5%|▌         | 1/20 [00:17<05:31, 17.43s/it]

40_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

________________________________________________________________________________________
22_training Model starting...

21_training.tif Loading...
Accuracy =  0.9669687234816341
Macro Avg: 0.86
21_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

22_training.tif Loading...
Accuracy =  0.9681688689538126
Macro Avg: 0.89
22_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

23_training.tif Loading...
Accuracy =  0.960025457631228
Macro Avg: 0.84
23_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

24_training.tif Loading...
Accuracy =  0.9481603830767366
Macro Avg: 0.85
24_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

25_training.tif Loading...
Accuracy =  0.9211116498969572
Macro Avg: 0.75
25_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

26_training.tif Loading...
Accuracy =  0.5035731603830768
Macro Avg: 0.43
26_training.tif Finishe

 10%|█         | 2/20 [00:35<05:14, 17.47s/it]

...

40_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

________________________________________________________________________________________
39_training Model starting...

21_training.tif Loading...
Accuracy =  0.9602830646138926
Macro Avg: 0.82
21_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

22_training.tif Loading...
Accuracy =  0.9490120014547218
Macro Avg: 0.82
22_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

23_training.tif Loading...
Accuracy =  0.9572311795369136
Macro Avg: 0.82
23_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

24_training.tif Loading...
Accuracy =  0.9366014062310584
Macro Avg: 0.81
24_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

25_training.tif Loading...
Accuracy =  0.9374105952236634
Macro Avg: 0.77
25_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

26_training.tif Loading...
Accuracy =  0.9133349496908716
Macro Avg: 0.51
26_training.tif Finish

 95%|█████████▌| 19/20 [05:27<00:17, 17.28s/it]

40_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

________________________________________________________________________________________
40_training Model starting...

21_training.tif Loading...
Accuracy =  0.9312886410473997
Macro Avg: 0.64
21_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

22_training.tif Loading...
Accuracy =  0.9378651957813068
Macro Avg: 0.75
22_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

23_training.tif Loading...
Accuracy =  0.9442326342586981
Macro Avg: 0.71
23_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

24_training.tif Loading...
Accuracy =  0.8954206570493393
Macro Avg: 0.63
24_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

25_training.tif Loading...
Accuracy =  0.9263516789913929
Macro Avg: 0.68
25_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

26_training.tif Loading...
Accuracy =  0.9523578615589768
Macro Avg: 0.83
26_training.tif Finish

100%|██████████| 20/20 [05:45<00:00, 17.27s/it]

40_training.tif Finished...
* * * * * * * * * * * * * * * * * * * *

________________________________________________________________________________________


In [39]:
len(acc_total)

21

In [40]:
col_list = ["Images"] + model_ids
col_list

['Images',
 '21_training',
 '22_training',
 '23_training',
 '24_training',
 '25_training',
 '26_training',
 '27_training',
 '28_training',
 '29_training',
 '30_training',
 '31_training',
 '32_training',
 '33_training',
 '34_training',
 '35_training',
 '36_training',
 '37_training',
 '38_training',
 '39_training',
 '40_training']

In [41]:
from numpy import transpose

In [42]:
modified = transpose(acc_total)
modified

array([['21_training', '0.91', '0.86', '0.83', '0.88', '0.75', '0.62',
        '0.87', '0.87', '0.59', '0.60', '0.82', '0.88', '0.86', '0.76',
        '0.80', '0.79', '0.79', '0.78', '0.82', '0.64'],
       ['22_training', '0.66', '0.89', '0.86', '0.76', '0.84', '0.72',
        '0.82', '0.80', '0.75', '0.75', '0.81', '0.85', '0.66', '0.77',
        '0.85', '0.85', '0.84', '0.51', '0.82', '0.75'],
       ['23_training', '0.75', '0.84', '0.91', '0.74', '0.83', '0.69',
        '0.81', '0.79', '0.67', '0.70', '0.82', '0.79', '0.73', '0.76',
        '0.82', '0.79', '0.84', '0.63', '0.82', '0.71'],
       ['24_training', '0.86', '0.85', '0.82', '0.90', '0.74', '0.62',
        '0.86', '0.86', '0.57', '0.62', '0.81', '0.87', '0.86', '0.76',
        '0.80', '0.77', '0.79', '0.83', '0.81', '0.63'],
       ['25_training', '0.55', '0.75', '0.79', '0.56', '0.86', '0.59',
        '0.71', '0.69', '0.71', '0.68', '0.70', '0.74', '0.55', '0.62',
        '0.76', '0.77', '0.75', '0.41', '0.77', '0.68'],


# Visualizing The DataSet

In [43]:
acc_df = pd.DataFrame(transpose(acc_total), columns=col_list)
acc_df

,Images,21_training,22_training,23_training,24_training,25_training,26_training,27_training,28_training,29_training,...,31_training,32_training,33_training,34_training,35_training,36_training,37_training,38_training,39_training,40_training
0,21_training,0.91,0.86,0.83,0.88,0.75,0.62,0.87,0.87,0.59,...,0.82,0.88,0.86,0.76,0.80,0.79,0.79,0.78,0.82,0.64
1,22_training,0.66,0.89,0.86,0.76,0.84,0.72,0.82,0.80,0.75,...,0.81,0.85,0.66,0.77,0.85,0.85,0.84,0.51,0.82,0.75
2,23_training,0.75,0.84,0.91,0.74,0.83,0.69,0.81,0.79,0.67,...,0.82,0.79,0.73,0.76,0.82,0.79,0.84,0.63,0.82,0.71
3,24_training,0.86,0.85,0.82,0.90,0.74,0.62,0.86,0.86,0.57,...,0.81,0.87,0.86,0.76,0.80,0.77,0.79,0.83,0.81,0.63
4,25_training,0.55,0.75,0.79,0.56,0.86,0.59,0.71,0.69,0.71,...,0.70,0.74,0.55,0.62,0.76,0.77,0.75,0.41,0.77,0.68
5,26_training,0.49,0.43,0.43,0.49,0.81,0.88,0.59,0.43,0.84,...,0.53,0.59,0.51,0.74,0.80,0.44,0.54,0.49,0.51,0.83
6,27_training,0.86,0.86,0.84,0.84,0.80,0.65,0.89,0.87,0.62,...,0.83,0.86,0.80,0.76,0.83,0.82,0.82,0.63,0.83,0.66
7,28_training,0.87,0.86,0.84,0.86,0.79,0.67,0.87,0.90,0.63,...,0.82,0.87,0.82,0.79,0.82,0.82,0.81,0.69,0.82,0.68
8,29_training,0.49,0.57,0.55,0.50,0.79,0.77,0.57,0.49,0.89,...,0.55,0.61,0.49,0.78,0.77,0.58,0.64,0.48,0.64,0.83
9,30_training,0.49,0.51,0.43,0.48,0.71,0.77,0.62,0.50,0.83,...,0.53,0.58,0.50,0.67,0.64,0.42,0.47,0.49,0.50,0.78


## Object To Float Conversion

In [44]:
dataType = acc_df.dtypes
dataType

Images         object
21_training    object
22_training    object
23_training    object
24_training    object
25_training    object
26_training    object
27_training    object
28_training    object
29_training    object
30_training    object
31_training    object
32_training    object
33_training    object
34_training    object
35_training    object
36_training    object
37_training    object
38_training    object
39_training    object
40_training    object
dtype: object

In [45]:
# get column names expects date column
train_cols = acc_df.columns.difference(['Images'])

# set those cols as object type
acc_df[train_cols] = acc_df[train_cols].astype(float)


In [46]:
dataType = acc_df.dtypes
dataType

Images          object
21_training    float64
22_training    float64
23_training    float64
24_training    float64
25_training    float64
26_training    float64
27_training    float64
28_training    float64
29_training    float64
30_training    float64
31_training    float64
32_training    float64
33_training    float64
34_training    float64
35_training    float64
36_training    float64
37_training    float64
38_training    float64
39_training    float64
40_training    float64
dtype: object

In [47]:
import seaborn as sns

## The Mean Graph

In [48]:
acc_df.mean().sort_values()

38_training    0.5860
33_training    0.6680
21_training    0.6785
26_training    0.6880
29_training    0.7080
24_training    0.7115
30_training    0.7135
28_training    0.7145
31_training    0.7195
40_training    0.7205
36_training    0.7410
27_training    0.7465
23_training    0.7480
34_training    0.7545
37_training    0.7565
39_training    0.7575
22_training    0.7630
32_training    0.7640
25_training    0.7840
35_training    0.8010
dtype: float64

In [49]:
acc_df.var().sort_values()

34_training    0.002310
35_training    0.003925
25_training    0.005025
40_training    0.008016
26_training    0.008269
30_training    0.010361
39_training    0.011093
37_training    0.011634
32_training    0.012446
29_training    0.013448
27_training    0.013771
31_training    0.014047
36_training    0.017136
22_training    0.018359
23_training    0.020017
33_training    0.021406
24_training    0.021582
38_training    0.023615
28_training    0.024816
21_training    0.025413
dtype: float64

In [50]:
var_df = (acc_df.var()*100).to_frame()
var_df.columns  = ["Variance"]
var_df

,Variance
21_training,2.541342
22_training,1.835895
23_training,2.001684
24_training,2.158184
25_training,0.502526
26_training,0.826947
27_training,1.377132
28_training,2.481553
29_training,1.344842
30_training,1.036079


In [51]:
var_df = str(acc_df.var())

In [66]:
var_df

'21_training    0.027206\n22_training    0.024232\n23_training    0.022927\n24_training    0.021908\n25_training    0.007406\n26_training    0.011205\n27_training    0.031919\n28_training    0.034658\n29_training    0.016925\n30_training    0.013542\n31_training    0.029969\n32_training    0.022794\n33_training    0.031848\n34_training    0.004742\n35_training    0.006034\n36_training    0.017885\n37_training    0.011234\n38_training    0.028325\n39_training    0.011906\n40_training    0.011940\ndtype: float64'

As we have 20 images, an ideal division would be 15-5, but due to the nature of the images we cant take any 15 images, alo the job could be done in less than that also